**Aeronautics Institute of Technology – ITA**

**Computer Vision – CM-203**

**Professors:** 

Marcos Ricardo Omena de Albuquerque Maximo

Gabriel Adriano de Melo


**Instructions:**

Before submitting your lab, be sure that everything is running correctly (in sequence): first, **restart the kernel** (`Runtime->Restart Runtime` in Colab or `Kernel->Restart` in Jupyter). Then, execute all cells (`Runtime->Run All` in Colab or `Cell->Run All` in Jupyter) and verifies that all cells run without any errors, expecially the automatic grading ones, i.e. the ones with `assert`s.

**Do not delete the answer cells**, i.e. the ones that contains `WRITE YOUR CODE HERE` or `WRITE YOUR ANSWER HERE`, because they contain metadata with the ids of the cells for the grading system. For the same reason, **do not delete the test cells**, i.e. the ones with `assert`s. The autograding system executes all the code sequentially, adding extra tests in the test cells. There is no problem in creating new cells, as long as you do not delete answer or test cells. Moreover, keep your solutions within the reserved spaces.

The notebooks are implemented to be compatible with Google Colab, and they install the dependencies and download the datasets automatically. The commands which start with ! (exclamation mark) are bash commands and can be executed in a Linux terminal.

---

## Installations and Configurations

The following cells install dependencies and configure the notebook for the implementation of the laboratory.

In [ ]:
# This cell installs the dependencies

!pip install numpy matplotlib

In [ ]:
# This cell imports the needed libraries

import numpy as np
import matplotlib.pyplot as plt
from math import pi, sin, cos, inf

In [ ]:
# This cell configures matplotlib

plt.rcParams.update({'font.size': 14})

In [ ]:
# This cell defines a function for resetting the random seed

def reset_seed():
    np.random.seed(42) # 42 is the answer to the Ultimate Question of Life, the Universe, and Everything

## Implementation of the Stochastic Simulator

First, to better understand the mathematical model used by the Kalman filter, you will implement the simulator using a stochastic dynamical system. Consider the equations in the state-space format as shown in class:
\begin{equation}
\mathbf{x}_t = \mathbf{A} \mathbf{x}_{t-1} + \mathbf{B} \mathbf{u}_t + \mathbf{w}_t,
\end{equation}
\begin{equation}
\mathbf{z}_t = \mathbf{C} \mathbf{x}_t + \mathbf{v}_t,
\end{equation}
where $\mathbf{A}$, $\mathbf{B}$ e $\mathbf{C}$ are matrices, $\mathbf{x}_t$ is the state in the time step (discrete instant) $t$, $\mathbf{u}_t$ is the control input relative to the time step $t$, and $\mathbf{w}_t$ e $\mathbf{v}_t$ are noises distributed such as $\mathbf{w}_t \sim \mathcal{N}(\mathbf{0}, \mathbf{Q})$ and $\mathbf{v}_t \sim \mathcal{N}(\mathbf{0}, \mathbf{R})$.
Notice that despite the Kalman filter considering that the dynamical system can be time-varying, i.e. the matrices that determine how the system changes with time are time-varying, we will consider a time-invariant system for the sake of simplicity.

In the following cell, we define the class `StochasticDynamicalSystemSimulator` to represent a stochastic dynamical system.

## Important note:

This laboratory follows a convention where all matrices are 2-dimensional NumPy arrays. Moreover, 1D vectors are 2-dimensional arrays with a single column. With this convention, matrix multiplication works as usual (following the Math/MATLAB convention) if you use the @ operator.

In [ ]:
class StochasticDynamicalSystemSimulator:
    """
    Represents a stochastic dynamical system simulator.
    """
    def __init__(self, A, B, C, Q, R, x0):
        """
        Initializes the simulator.
        :param A: the state transition matrix.
        :param B: the input matrix.
        :param C: the observation matrix.
        :param Q: the covariance matrix of the process noise.
        :param R: the covariance matrix of the observation noise.
        :param x0: the initial state of the simulation.
        """
        self.A = A
        self.B = B
        self.C = C
        self.Q = Q
        self.R = R
        self.x = x0

To allow simulations to test the implementation, we define matrices associated to a simple 1-dimensional kinematic dynamical system:
\begin{equation}
\mathbf{x}_t = 
\begin{bmatrix}
x_t \\ \dot{x}_t
\end{bmatrix},
\end{equation}
\begin{equation}
\mathbf{A} = 
\begin{bmatrix}
1 & T \\ 0 & 1
\end{bmatrix},
\end{equation}
\begin{equation}
\mathbf{B} = 
\begin{bmatrix}
T^2/2 \\ T,
\end{bmatrix},
\end{equation}
\begin{equation}
\mathbf{C} = 
\begin{bmatrix}
1 & 0
\end{bmatrix},
\end{equation}
where $T$ is the sample time. As shown in class, to determine $\mathbf{Q}$, we adopt an input as a Gaussian noise with null mean and covariance $\mathbf{M}$, so we compute
\begin{equation}
\mathbf{Q} = \mathbf{B} \mathbf{M} \mathbf{B}^T.
\end{equation}
Moreover, we have
\begin{equation}
\mathbf{M} = \sigma_{\mathrm{acc}}^2,
\end{equation}
\begin{equation}
\mathbf{R} = \sigma_{\mathrm{cam}}^2,
\end{equation}
where $\sigma_{\mathrm{acc}}$ and $\sigma_{\mathrm{cam}}$ are variances associated to the uncertainties in acceleration and observation, respectively.

In [ ]:
def get_system_1d(T, sigma_acc, sigma_cam):
    """
    Obtains a stochastic dynamical system based on kinematics equations along a single axis 
    whose states are 1D position and velocity, and the input is 1D acceleration. Moreover, the
    observation is the position.
    :return A: the state transition matrix.
    :return B: the input matrix.
    :return C: the observation matrix.
    :return Q: the covariance matrix of the process noise.
    :return R: the covariance matrix of the observation noise.
    """
    A = np.array([[1.0, T], [0.0, 1.0]])
    B = np.array([[T ** 2 / 2.0], [T]])
    C = np.array([[1.0, 0.0]])
    M = np.array([[sigma_acc ** 2]])
    Q = B @ M @ B.T
    R = np.array([[sigma_cam ** 2]])
    return A, B, C, Q, R

In the following, implement the method `step()` (from class `StochasticDynamicalSystemSimulator`), which executes a simulation step accordingly to
\begin{equation}
\mathbf{x}_t = \mathbf{A} \mathbf{x}_{t-1} + \mathbf{B} \mathbf{u}_t + \mathbf{w}_t.
\end{equation}
Hint: to generate the noise $\mathbf{w}_t$, use the function `np.random.multivariate_normal()` from NumPy. Considering the conventions adopted in the laboratory, we need to call the function as:

`np.random.multivariate_normal(mean, covariance, 1).T`,

where `mean` is the mean (a 1-dimensioanl `np.array` with zeros), `covariance` is the covariance matrix (in our case, $\mathbf{Q}$) and the last argument indicates that a single random sample will be generated.

In [ ]:
def step(self, u : np.ndarray):
    """
    Steps the simulation.
    :param u: the control input.
    :return: updated state after the step.
    """
    # WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
    raise NotImplementedError()
    return self.x

# Adding the step() method to the StochasticDynamicalSystemSimulator class
StochasticDynamicalSystemSimulator.step = step

In [ ]:
T = 0.1
sigma_acc = 2.0
sigma_cam = 0.9
x0 = np.array([[0.1], [1.0]])
u = np.array([[1.0]])
reset_seed()
A, B, C, Q, R = get_system_1d(T, sigma_acc, sigma_cam)

simulator = StochasticDynamicalSystemSimulator(A, B, C, Q, R, x0)
x = simulator.step(u)
print(x)
assert np.max(np.abs(x - np.array([[0.20003286], [1.00065717]]))) < 1e-3

The following code executes 100 Monte Carlo simulations, executing 100 simulation steps for each Monte Carlo simulation. Therefore, we can visualize the evolution of the stochastic dynamical system. Moreover, the 1-dimesional kinematic model is used.

In [ ]:
T = 0.1
sigma_acc = 2.0
sigma_cam = 0.9
x0 = np.array([[0.1], [1.0]])
u = np.array([[1.0]])
reset_seed()
A, B, C, Q, R = get_system_1d(T, sigma_acc, sigma_cam)

num_mc = 100
num_steps = 100
time = T * np.arange(num_steps) + T
x_history = np.zeros((A.shape[0], num_steps))
plt.figure()
for m in range(num_mc):
    simulator = StochasticDynamicalSystemSimulator(A, B, C, Q, R, x0)
    for i in range(num_steps):
        x_history[:, i] = simulator.step(u).T
    plt.plot(time, x_history[0, :])
plt.xlabel('Time (s)')
plt.ylabel('Position (m)')
plt.title('Monte Carlo Simulation')
plt.grid()

In the following, implement the method `observe()` (from class `StochasticDynamicalSystemSimulator`), which generates an observation accordingly to
\begin{equation}
\mathbf{z}_t = \mathbf{C} \mathbf{x}_t + \mathbf{v}_t.
\end{equation}

In [ ]:
def observe(self):
    """
    Obtains an observation.
    :return: an observation of the system.
    """
    # WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
    raise NotImplementedError()
    return z

# Adding the observe() method to the StochasticDynamicalSystemSimulator class
StochasticDynamicalSystemSimulator.observe = observe

In [ ]:
T = 0.1
sigma_acc = 2.0
sigma_cam = 0.9
x0 = np.array([[0.1], [1.0]])
u = np.array([[1.0]])
z = np.array([[3.0]])
A, B, C, Q, R = get_system_1d(T, sigma_acc, sigma_cam)
reset_seed()

simulator = StochasticDynamicalSystemSimulator(A, B, C, Q, R, x0)
z = simulator.observe()
assert np.max(np.abs(z - np.array([[0.54704274]]))) < 1e-3
print(z)

The following cell generates 10000 observations. Then, it shows a histogram of the measurements, together with their average and variance.

In [ ]:
T = 0.1
sigma_acc = 2.0
sigma_cam = 0.9
x0 = np.array([[0.1], [1.0]])
z = np.array([[3.0]])
A, B, C, Q, R = get_system_1d(T, sigma_acc, sigma_cam)
reset_seed()

num_steps = 10000
z_history = np.zeros((C.shape[0], num_steps))
plt.figure()
simulator = StochasticDynamicalSystemSimulator(A, B, C, Q, R, x0)
for i in range(num_steps):
    z_history[:, i] = simulator.observe()
plt.plot(z_history[0, :])
plt.plot(np.mean(z_history[0, :]) * np.ones_like(z_history[0, :]), '--')
plt.grid()
plt.xlabel('Iteration (-)')
plt.ylabel('Observation (m)')
plt.title('Observation Simulation')

plt.figure()
plt.grid()
plt.hist(z_history[0, :], 100)
plt.xlabel('Position (m)')
plt.ylabel('Frequency (-)')
plt.title('Histogram of Observations')

print('Mean: ', np.mean(z_history[0, :]))
print('Std: ', np.std(z_history[0, :]))

In the following, we define a Kalman filter. The Kalman filter consists of two steps: prediction and filtering. The prediction step is given by the following equations:
\begin{equation}
\hat{\mathbf{x}}^-_t = \mathbf{A} \hat{\mathbf{x}}_{t-1}^+ + \mathbf{B} \mathbf{u}_t,
\end{equation}
\begin{equation}
\mathbf{P}^-_t = \mathbf{A} \mathbf{P}^+_{t-1} \mathbf{A}^T + \mathbf{Q},
\end{equation}
where $\hat{\mathbf{x}}^-_t$ e $\mathbf{P}^-_t$ represent, respectively, the mean and the covariance of the estimate in the time step $t$, before incorporating the observation. Then, the filtering step is given by the following equations:
\begin{equation}
\mathbf{K}_t = \mathbf{P}^-_t \mathbf{C}^T (\mathbf{R} + \mathbf{C} \mathbf{P}^-_t \mathbf{C}^T)^{-1},
\end{equation}
\begin{equation}
\hat{\mathbf{x}}^+_t = \hat{\mathbf{x}}^-_t + \mathbf{K}_t (\mathbf{z}_t - \mathbf{C} \hat{\mathbf{x}}^-_t),
\end{equation}
\begin{equation}
\mathbf{P}^+_t = \mathbf{P}^-_t - \mathbf{K}_t \mathbf{C} \mathbf{P}^-_t,
\end{equation}
where $\hat{\mathbf{x}}^+_t$ e $\mathbf{P}^+_t$ are, respectively, the average and the covariance of the estimate in the time step $t$, after incorporating the observation.

In [ ]:
class KalmanFilter:
    """
    This class represents a Kalman filter. It considers a 
    time-invariant stochastic dynamical system as its model.
    """
    def __init__(self, A, B, C, Q, R, x0, P0):
        """
        Initializes the Kalman filter.
        :param A: the state transition matrix.
        :param B: the input matrix.
        :param C: the observation matrix.
        :param Q: the covariance matrix of the process noise.
        :param R: the covariance matrix of the observation noise.
        :param x0: the initial mean of the estimate.
        :param P0: the initial covariance matrix of the estimate.
        """
        self.A = A
        self.B = B
        self.C = C
        self.Q = Q
        self.R = R
        self.reset(x0, P0)
        
    def reset(self, x0, P0):
        """
        Resets the filter.
        :param x0: the new mean of the estimate after the reset.
        :param P0: the covariance matrix of the estimate after the reset.
        """
        self.x = x0
        self.P = P0

Implement the method `predict()`, which executes a prediction step of the Kalman filter.

In [ ]:
def predict(self, u : np.ndarray):
    """
    Executes a prediction step.
    :param u: the control input.
    :return x: updated mean of the estimate.
    :return P: updated covariance matrix of the estimate.
    """
    # WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
    raise NotImplementedError()
    return self.x, self.P

# Adding the predict() method to the KalmanFilter class
KalmanFilter.predict = predict

In [ ]:
T = 0.1
sigma_acc = 2.0
sigma_cam = 0.9
x0 = np.array([[0.1], [1.0]])
P0 = np.identity(2)
u = np.array([[1.0]])
A, B, C, Q, R = get_system_1d(T, sigma_acc, sigma_cam)
reset_seed()

filter = KalmanFilter(A, B, C, Q, R, x0, P0)
filter.predict(u)
print(filter.x)
print(filter.P)
assert np.max(np.abs(filter.x - np.array([[0.205], [1.1]]))) < 1e-3
assert np.max(np.abs(filter.P - np.array([[1.0101, 0.102], [0.102,  1.04]]))) < 1e-3

The following code executes an experiment in which the Kalman filter is initialized with the same state as the simulator, but the estimate is updated only with prediction steps. Therefore, the estimate tends to diverge from the real state through time, since there are no corrections from observations. This phenomenon is the so-called *drift*.

In [ ]:
T = 0.1
sigma_acc = 2.0
sigma_cam = 0.9
x0 = np.array([[0.1], [1.0]])
P0 = np.identity(2)
u = np.array([[1.0]])
A, B, C, Q, R = get_system_1d(T, sigma_acc, sigma_cam)
reset_seed()

num_steps = 100
time = T * np.arange(num_steps) + T
x_history = np.zeros((A.shape[0], num_steps))
z_history = np.zeros((C.shape[0], num_steps))
xhat_history = np.zeros((A.shape[0], num_steps))
simulator = StochasticDynamicalSystemSimulator(A, B, C, Q, R, x0)
filter = KalmanFilter(A, B, C, Q, R, x0, P0)
for i in range(num_steps):
    x_history[:, i] = simulator.step(u).T
    z_history[:, i] = simulator.observe()
    xhat, _ = filter.predict(u)
    xhat_history[:, i] = xhat.T
    
plt.figure()
plt.plot(time, x_history[0, :])
plt.plot(time, xhat_history[0, :])
plt.plot(time, z_history[0, :], '.')
plt.grid()
plt.xlabel('Time (s)')
plt.ylabel('Position (m)')
plt.title('Position Filtering (Prediction Only)')
plt.legend(['Ground Truth', 'Estimate', 'Observation'])

plt.figure()
plt.plot(time, x_history[1, :])
plt.plot(time, xhat_history[1, :])
plt.grid()
plt.xlabel('Time (s)')
plt.ylabel('Velocity (m/s)')
plt.title('Velocity Filtering (Prediction Only)')
plt.legend(['Ground Truth', 'Estimate'])

In the following, implement the method `filter()`, which executes a filtering step of the Kalman filter.

In [ ]:
def filter(self, z):
    """
    Executes a filtering step.
    :param z: observation.
    :return x: updated mean of the estimate.
    :return P: updated covariance matrix of the estimate.
    """
    # WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
    raise NotImplementedError()
    return self.x, self.P

# Adding the filter() method to the KalmanFilter class
KalmanFilter.filter = filter

In [ ]:
T = 0.1
sigma_acc = 2.0
sigma_cam = 0.9
x0 = np.array([[0.1], [1.0]])
P0 = np.identity(2)
z = np.array([[3.0]])
A, B, C, Q, R = get_system_1d(T, sigma_acc, sigma_cam)
reset_seed()

filter = KalmanFilter(A, B, C, Q, R, x0, P0)
filter.filter(z)
print(filter.x)
print(filter.P)
assert np.max(np.abs(filter.x - np.array([[1.70220994], [1.0]]))) < 1e-3
assert np.max(np.abs(filter.P - np.array([[0.44751381, 0.0], [0.0,  1.0]]))) < 1e-3

In the following, we execute a complete simulation of the Kalman filter, considering the prediction and filtering steps. In this case, we consider that the filter has access to the control input $\mathbf{u}_t$. The estimate of the velocity observation is obtained through numeric differentiation:
\begin{equation}
\dot{x}_t \approx \frac{x_t - x_{t-1}}{T}.
\end{equation}
Notice that the Kalman filter has a performance much superior regarding velocity estimate, when comparing with the numeric differentiation. As discussed in class, differentiation tends to amplify noise.

In [ ]:
T = 0.1
sigma_acc = 2.0
sigma_cam = 0.9
x0 = np.array([[0.1], [1.0]])
x0_filter = np.array([[0.0], [0.0]]) # so the filter starts with a wrong estimate
P0 = np.identity(2)
u = np.array([[1.0]])
A, B, C, Q, R = get_system_1d(T, sigma_acc, sigma_cam)
reset_seed()

num_steps = 100
time = T * np.arange(num_steps) + T
x_history = np.zeros((A.shape[0], num_steps))
z_history = np.zeros((C.shape[0], num_steps))
dz_history = np.zeros((C.shape[0], num_steps))
xhat_history = np.zeros((A.shape[0], num_steps))
simulator = StochasticDynamicalSystemSimulator(A, B, C, Q, R, x0)
filter = KalmanFilter(A, B, C, Q, R, x0_filter, P0)
for i in range(num_steps):
    x_history[:, i] = simulator.step(u).T
    z_history[:, i] = simulator.observe()
    filter.predict(u)
    xhat, _ = filter.filter(z_history[:, i])
    xhat_history[:, i] = xhat.T

plt.figure()
plt.plot(time, x_history[0, :])
plt.plot(time, xhat_history[0, :])
plt.plot(time, z_history[0, :], '.', zorder=-1)
dz_history[0, 1:] = (1.0 / T) * np.diff(z_history[0, :])
plt.grid()
plt.xlabel('Time (s)')
plt.ylabel('Position (m)')
plt.title('Position Filtering')
plt.legend(['Ground Truth', 'Estimate', 'Observation'])

plt.figure()
plt.plot(time, x_history[1, :])
plt.plot(time, xhat_history[1, :])
plt.plot(time, dz_history[0, :], '.', zorder=-1)
plt.grid()
plt.xlabel('Time (s)')
plt.ylabel('Velocity (m/s)')
plt.title('Velocity Filtering')
plt.legend(['Ground Truth', 'Estimate', 'Observation'])

plt.figure()
plt.plot(time, x_history[1, :])
plt.plot(time, xhat_history[1, :])
plt.plot(time, dz_history[0, :], '.', zorder=-1)
plt.ylim(0, 10)
plt.grid()
plt.xlabel('Time (s)')
plt.ylabel('Velocity (m/s)')
plt.title('Velocity Filtering (Zoom)')
plt.legend(['Ground Truth', 'Estimate', 'Observation'])

As discussed in class, in practice, in a tracking problem, the tracking has no access to the control input, so we consider $\mathbf{u}_t=\mathbf{0}$. In the following, the same simulation cell as before is repeated, but assuming that the filter receives $\mathbf{u}_t=\mathbf{0}$ in the prediction step, which does not match the acceleration effectively executed by the target. This mismatch between the filter and reality makes the filter take more time to converge, when compared to the previous simulation, especially as observed in the last plot (velocity plot with *zoom*).

In [ ]:
T = 0.1
sigma_acc = 2.0
sigma_cam = 0.9
x0 = np.array([[0.1], [1.0]])
x0_filter = np.array([[0.0], [0.0]]) # so the filter starts with a wrong estimate
P0 = np.identity(2)
u = np.array([[1.0]])
A, B, C, Q, R = get_system_1d(T, sigma_acc, sigma_cam)
reset_seed()

num_steps = 100
time = T * np.arange(num_steps) + T
x_history = np.zeros((A.shape[0], num_steps))
z_history = np.zeros((C.shape[0], num_steps))
dz_history = np.zeros((C.shape[0], num_steps))
xhat_history = np.zeros((A.shape[0], num_steps))
simulator = StochasticDynamicalSystemSimulator(A, B, C, Q, R, x0)
filter = KalmanFilter(A, B, C, Q, R, x0_filter, P0)
for i in range(num_steps):
    x_history[:, i] = simulator.step(u).T
    z_history[:, i] = simulator.observe().T
    filter.predict(np.array([[0.0]]))
    xhat, _ = filter.filter(z_history[:, i])
    xhat_history[:, i] = xhat.T

plt.figure()
plt.plot(time, x_history[0, :])
plt.plot(time, xhat_history[0, :])
plt.plot(time, z_history[0, :], '.', zorder=-1)
dz_history[0, 1:] = (1.0 / T) * np.diff(z_history[0, :])
plt.grid()
plt.xlabel('Time (s)')
plt.ylabel('Position (m)')
plt.title('Position Tracking')
plt.legend(['Ground Truth', 'Estimate', 'Observation'])

plt.figure()
plt.plot(time, x_history[1, :])
plt.plot(time, xhat_history[1, :])
plt.plot(time, dz_history[0, :], '.', zorder=-1)
plt.grid()
plt.xlabel('Time (s)')
plt.ylabel('Velocity (m/s)')
plt.title('Velocity Tracking')
plt.legend(['Ground Truth', 'Estimate', 'Observation'])

plt.figure()
plt.plot(time, x_history[1, :])
plt.plot(time, xhat_history[1, :])
plt.plot(time, dz_history[0, :], '.', zorder=-1)
plt.ylim(-1, 10)
plt.grid()
plt.xlabel('Time (s)')
plt.ylabel('Velocity (m/s)')
plt.title('Velocity Tracking (Zoom)')
plt.legend(['Ground Truth', 'Estimate', 'Observation'])

As shown in class, in a tracking problem in a video, we usually use a 2D kinematic model, where the state vector contains position and velocity in axes $x$ e $y$:
\begin{equation}
\mathbf{x}_t = 
\begin{bmatrix}
x_t \\ y_t \\ \dot{x}_t \\ \dot{y}_t
\end{bmatrix}.
\end{equation}

Hence, the state space is given by
\begin{equation}
\mathbf{A} = 
\begin{bmatrix}
1 & 0 & T & 0 \\
0 & 1 & 0 & T \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix},
\end{equation}
\begin{equation}
\mathbf{B} = 
\begin{bmatrix}
\dfrac{T^2}{2} & 0 \\
0 & \dfrac{T^2}{2} \\
T & 0 \\
0 & T
\end{bmatrix},
\end{equation}
\begin{equation}
\mathbf{C} = 
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0
\end{bmatrix},
\end{equation}
\begin{equation}
\mathbf{M} =
\begin{bmatrix}
\sigma^2_{\mathrm{acc}} & 0 \\
0 & \sigma^2_{\mathrm{acc}}
\end{bmatrix},
\end{equation}
\begin{equation}
\mathbf{Q} =
\mathbf{B} \mathbf{M} \mathbf{B}^T,
\end{equation}
\begin{equation}
\mathbf{R} =
\begin{bmatrix}
\sigma^2_{\mathrm{cam}} & 0 \\
0 & \sigma^2_{\mathrm{cam}}
\end{bmatrix}.
\end{equation}

In [ ]:
def get_system_2d(T, sigma_acc, sigma_cam):
    """
    Obtains a stochastic dynamical system based on kinematics equations along a single axis 
    whose states are 2D position and velocity, and the input is 2D acceleration. Moreover, the
    observation is the 2D position.
    :return A: the state transition matrix.
    :return B: the input matrix.
    :return C: the observation matrix.
    :return Q: the covariance matrix of the process noise.
    :return R: the covariance matrix of the observation noise.
    """
    # WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
    raise NotImplementedError()
    return A, B, C, Q, R

In [ ]:
T = 0.1
sigma_acc = 2.0
sigma_cam = 0.9
x0 = np.array([[0.0],
               [0.0],
               [0.0],
               [0.0]])
P0 = np.identity(4)
u = np.array([[1.0],
              [1.0]])
A, B, C, Q, R = get_system_2d(T, sigma_acc, sigma_cam)
reset_seed()

simulator = StochasticDynamicalSystemSimulator(A, B, C, Q, R, x0)
x = simulator.step(u)
print(x)
assert np.max(np.abs(x - np.array([[3.28589601e-05], [3.61735584e-03], [6.57169373e-04], [7.23471398e-02]]))) < 1e-3
z = simulator.observe()
print(z)
assert np.max(np.abs(z - np.array([[-0.21070518], [-0.20710591]]))) < 1e-3

In the following experiment, we verify the performance of a 2D tracker based on the Kalman filter. We simulate 500 simulation steps with $T = 0.1 \ s$. During time steps $t \in [270, 300)$, we simulate loss of object detection (which may occur due to occlusion, for example), so the filter stays executing only prediction steps during this time. We point out that the filter is capable to maintain a resonable estimate until the detection resumes.

In [ ]:
T = 0.1
sigma_acc = 2.0
sigma_cam = 0.9
x0 = np.array([[0.0],
               [0.0],
               [0.0],
               [0.0]])
P0 = np.identity(4)
u = np.array([[1.0],
              [1.0]])
A, B, C, Q, R = get_system_2d(T, sigma_acc, sigma_cam)
reset_seed()

num_steps = 500
time = T * np.arange(num_steps) + T
x_history = np.zeros((A.shape[0], num_steps))
z_history = np.zeros((C.shape[0], num_steps))
dz_history = np.zeros((C.shape[0], num_steps))
xhat_history = np.zeros((A.shape[0], num_steps))
simulator = StochasticDynamicalSystemSimulator(A, B, C, Q, R, x0)
filter = KalmanFilter(A, B, C, Q, R, x0, P0)
for i in range(num_steps):
    t = i * T
    u[0, 0] = sin(t / (2.0 * pi) + pi / 2.0)
    u[1, 0] = sin(t / (2.0 * pi) * t)
    x_history[:, i] = simulator.step(u)[:, 0]
    z_history[:, i] = simulator.observe()[:, 0]
    x, _ = filter.predict(np.array([[0.0], [0.0]]))
    if i >= 270 and i < 300:
        z_history[:, i] = np.array([inf, inf])
    else:
        filter.filter(np.array([z_history[:, i]]).T)
    xhat_history[:, i] = (filter.x)[:, 0]
    
plt.figure()
plt.plot(x_history[0, :], x_history[1, :])
plt.plot(xhat_history[0, :], xhat_history[1, :])
plt.plot(z_history[0, :], z_history[1, :], '.', zorder=-1)
plt.grid()
plt.xlabel('X (m)')
plt.ylabel('Y (m)')
plt.title('X-Y view')
plt.legend(['Ground Truth', 'Estimate', 'Observation'])

plt.figure()
plt.plot(time, x_history[2, :])
plt.plot(time, xhat_history[2, :])
plt.grid()
plt.xlabel('Time (s)')
plt.ylabel('X Velocity (m/s)')
plt.title('X Velocity Tracking')
plt.legend(['Ground Truth', 'Estimate'])

plt.figure()
plt.plot(time, x_history[3, :])
plt.plot(time, xhat_history[3, :])
plt.grid()
plt.xlabel('Time (s)')
plt.ylabel('Y Velocity (m/s)')
plt.title('Y Velocity Tracking')
plt.legend(['Ground Truth', 'Estimate'])

# Your data and feedback:

Write a feedback for the lab so we can make it better for the next years.

In the following variables, write the number of hours spent on this lab, the perceived difficulty, and the expected grade (you may delete the `raise` and the comments):

In [ ]:
# meta_eval manual_graded_answer 0

horas_gastas = None    # 1.5   - Float number with the number of hours spent 
dificuldade_lab = None # 0     - Float number from 0.0 to 10.0 (inclusive)
nota_esperada = None   # 10    - Float number from 0.0 to 10.0 (inclusive)

# WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
raise NotImplementedError()

Write below other comments or feedbacks about the lab. If you did not understand anything about the lab, please also comment here.

If you find any typo or bug in the lab, please comment below so we can fix it.

WRITE YOUR SOLUTION HERE! (do not change this first line):

**ATTENTION**

**ATTENTION**

**ATTENTION**

**ATTENTION**

**DISCURSIVE QUESTION**

WRITE YOUR ANSWER HERE (do not delete this cell so the ID is not lost)

**ATTENTION**

**ATTENTION**


**End of the lab!**